In [11]:
"""Daisy's Forecast Risk Metrics
======================================

Forecast Models
---------------
• 1-Day & 5-Day EWMA Volatility
• 1-Day GARCH(1,1) Volatility
• 1-Day GARCH(1,1) VaR
• 1-Day GARCH(1,1) CVaR

Assumptions
-----------
• Portfolio return series = proxy-adjusted daily return stream  
• All models applied only on portfolio-level return  
"""

from __future__ import annotations
import pandas as pd, numpy as np, yfinance as yf
import logging, contextlib, io, os
import matplotlib.pyplot as plt
import warnings
from matplotlib.ticker import FuncFormatter
from arch import arch_model
from pathlib import Path


# ────────── 1) USER SETTINGS ──────────
YEARS           = 5
PORTF_VALUE_USD = 100_000
CONF_LVL        = 0.95
LAMBDA          = 0.94  # EWMA decay
ROLLING_WINDOW  = 252

CSV_PATH        = os.path.join("..", "data", "portfolio_weights.csv")
PROXY_CSV_PATH  = os.path.join("..", "data", "proxy_mapping.csv")

# ────────── 2) LOAD WEIGHTS & PROXIES ──────────
if not os.path.isfile(CSV_PATH):
    raise FileNotFoundError(f"Portfolio file not found: {CSV_PATH}")

df = pd.read_csv(CSV_PATH, dtype={"Ticker": str, "Weight": float})
df["Ticker"] = df["Ticker"].str.strip()
df = df.groupby("Ticker", as_index=False)["Weight"].sum()
WEIGHTS = dict(zip(df["Ticker"], df["Weight"]))

BENCH_SPY, BENCH_QQQ = "SPY", "QQQ"

if not os.path.isfile(PROXY_CSV_PATH):
    raise FileNotFoundError(f"Proxy file not found: {PROXY_CSV_PATH}")

proxy_df = pd.read_csv(PROXY_CSV_PATH)
PROXY = {target: list(zip(g["Proxy"], g["Weight"])) for target, g in proxy_df.groupby("Target")}

# ────────── 3) HELPERS ──────────
logging.getLogger("yfinance").setLevel(logging.CRITICAL)
TRADING_DAYS_YR = 252

@contextlib.contextmanager
def silence():
    with contextlib.redirect_stdout(io.StringIO()):
        yield

def adj_close(df: pd.DataFrame, t: str) -> pd.Series:
    close = df.xs("Close", level=0, axis=1).squeeze("columns") if isinstance(df.columns, pd.MultiIndex) else df["Close"]
    close.name = t
    return close

# ────────── 4) DOWNLOAD PRICES ──────────
window = int(YEARS * TRADING_DAYS_YR)
start  = pd.Timestamp.today().normalize() - pd.tseries.offsets.BDay(window + 40)
end    = pd.Timestamp.today().normalize()

proxy_members = {m for v in PROXY.values() for m in ([v] if isinstance(v, str) else [x for x, _ in v])}
universe = set(WEIGHTS) | {BENCH_SPY, BENCH_QQQ} | proxy_members

prices: dict[str, pd.Series] = {}
for tic in universe:
    with silence():
        raw = yf.download(tic, start=start, end=end, auto_adjust=True, progress=False, threads=False)
    if raw.empty:
        if WEIGHTS.get(tic, 0) > 0:
            raise RuntimeError(f"Missing data for funded ticker {tic}")
        continue
    prices[tic] = adj_close(raw, tic)

df = pd.concat(prices.values(), axis=1)
qqq_series = df[BENCH_QQQ].ffill()

# ────────── 5) PROXY FILL ──────────
def proxy_ser(spec):
    return df.get(spec, qqq_series) if isinstance(spec, str) else pd.concat([df.get(t, qqq_series)*w for t, w in spec], axis=1).sum(axis=1)

filled = []
for tic, w in WEIGHTS.items():
    base = df.get(tic, pd.Series(index=df.index, dtype=float, name=tic))
    prox = proxy_ser(PROXY.get(tic, BENCH_QQQ))
    first = base.first_valid_index()
    combined = prox if first is None else base.combine_first(prox.where((base.index < first) & base.isna()))
    filled.append(combined)

prices_clean = pd.concat(filled, axis=1)
prices_clean = prices_clean.loc[:, ~prices_clean.columns.duplicated()]
prices_clean = prices_clean.dropna().tail(window + 1)

# ────────── 6) RETURNS ──────────
rets = prices_clean.pct_change().dropna()
active_weights = {t: w for t, w in WEIGHTS.items() if t in rets.columns and w > 0}
portfolio_r = rets[list(active_weights)] @ pd.Series(active_weights)

# ────────── 7) FORECAST RISK METRICS ──────────

# ────────── 1) EWMA VOLATILITY ──────────

# EWMA variance and vol (1-day)
ewma_var = portfolio_r.ewm(alpha=1 - lambda_, adjust=False).var(bias=False)
ewma_vol_1d = np.sqrt(ewma_var.iloc[-1])

# Scale to 5-day using sqrt(5)
ewma_vol_5d = ewma_vol_1d * np.sqrt(5)

# Scale to 20-day using sqrt(20)
ewma_vol_20d = ewma_vol_1d * np.sqrt(20)

# ────────── 2) GARCH(1,1) FORECAST ──────────
garch_model = arch_model(portfolio_r * 100, mean='Zero', vol='GARCH', p=1, q=1, dist='normal')
garch_fit = garch_model.fit(disp="off")

# Forecast 1-day volatility (percent scale)
garch_forecast = garch_fit.forecast(horizon=1, reindex=False)
garch_vol_1d_pct = np.sqrt(garch_forecast.variance.values[-1, 0])
garch_vol_1d = garch_vol_1d_pct / 100  # Convert back to decimal scale

# VaR and CVaR from GARCH (1-day, 95%)
garch_var = -norm.ppf(CONF_LVL) * garch_vol_1d
garch_cvar = (1 / (1 - CONF_LVL)) * norm.pdf(norm.ppf(CONF_LVL)) * garch_vol_1d

# ────────── 3) PRINT METRICS ──────────
print(f"\nForecast Risk Metrics ({YEARS} Years)\n")
print(f"EWMA 1-Day Volatility    : {ewma_vol_1d:.2%}")
print(f"EWMA 5-Day Volatility    : {ewma_vol_5d:.2%}")
print(f"EWMA 20-Day Volatility   : {ewma_vol_20d:.2%}")
print(f"GARCH(1,1) 1-Day Volatility : {garch_vol_1d:.2%}")
print(f"GARCH(1,1) 1-Day VaR ({int(CONF_LVL*100)}%) : {-garch_var:.2%} (~${-garch_var*PORTF_VALUE_USD:,.0f})")
print(f"GARCH(1,1) 1-Day CVaR ({int(CONF_LVL*100)}%): {-garch_cvar:.2%} (~${-garch_cvar*PORTF_VALUE_USD:,.0f})")
print("\n❤️ DAISY ❤️")

# ────────── 8) ROLLING FORECAST RISK METRICS ──────────



Forecast Risk Metrics (5 Years)

EWMA 1-Day Volatility    : 1.06%
EWMA 5-Day Volatility    : 2.37%
EWMA 20-Day Volatility   : 4.73%
GARCH(1,1) 1-Day Volatility : 1.19%
GARCH(1,1) 1-Day VaR (95%) : 1.95% (~$1,949)
GARCH(1,1) 1-Day CVaR (95%): -2.44% (~$-2,445)

❤️ DAISY ❤️
